In [2]:
import os


def result_file_name(args):
    file_name = f"results_{args.fold_range}_{args.seed}"
    if len(args.modalities) > 0:
        file_name += "_"
        file_name += "_".join(args.modalities)
    return file_name

class Args:
    def __init__(self, modalities=["clinical", "miRNA", "mRNA", "WSI"]):
        self.epochs = 20
        self.modality_data_path = {'clinical': './preprocess/preprocessed_data/clinical_kidney.csv',
                                    'mRNA': './preprocess/preprocessed_data/mrna_kidney.csv',
                                    'miRNA': './preprocess/preprocessed_data/mirna_kidney.csv',
                                    'WSI': './preprocess/preprocessed_data/UNI2_features/TCGA-Kidney.pt'}
        self.device = "cuda"
        self.modalities = modalities
        self.input_modality_dim = {'clinical':4, 'mRNA':2746, 'miRNA':743 , 'WSI': 1536}
        self.fold_range = 5
        self.fold = 1
        self.modality_fv_len = 128
        self.batch_size = 128
        self.learning_rate = 1e-4
        self.loss_trade_off = 0.3
        self.loss_mode = 'total'
        self.seed = 24
        self.result_path = f"./logs/max/{result_file_name(self)}"
        print(f"Result Path: {self.result_path}")
        self.log_file_name = "temp.log"
        self.scheduler_patience = 5
        self.num_workers = 4
        self.num_modalities = len(self.modalities)
        self.split_path = "./splits/kidney_splits"
        self.remove_missing = True

args = Args()

Result Path: ./logs/max/results_5_24_clinical_miRNA_mRNA_WSI


In [3]:
import torch
from torch import nn
import torch.nn.functional as F


class AggModel(nn.Module):
    def __init__(self, n_views, n_in_feats, encoder_model):
        super(AggModel, self).__init__()
        self.modality_count = n_views
        self.modality_fv_len = n_in_feats
        
        self.encoder_model = encoder_model
        self.hazard_layer = nn.Linear(self.modality_fv_len, 1)
        self.label_layer = nn.Linear(self.modality_fv_len, 2)
			
    def forward(self, x_modality, mask):
        representation = self.encoder_model(x_modality)
        missing_rep = []
        for i, rep in enumerate(representation):
            index = torch.ones((rep.shape[0]), dtype=int) * i 
            index = index.to(rep.device)
            modality_mask = mask[:, i].reshape(-1, 1)
            rep =  modality_mask * rep
            missing_rep.append(rep)
        
        representation_dict = {} #used for self supervised loss
        for i, data in enumerate(missing_rep):
            representation_dict[i] = data

        final_representation = torch.cat(missing_rep, 1).reshape(-1, self.modality_count, self.modality_fv_len)
        final_representation = torch.max(final_representation, 1)[0]
        
        hazard = self.hazard_layer(final_representation)
        score = F.log_softmax(self.label_layer(final_representation), dim=1)
        return {'hazard':hazard, 'score':score}, representation_dict

In [4]:
from lifelines.utils import concordance_index


def print_log(msg, logger):
    print(msg)
    logger.info(msg)

def evaluation(model, dataloader, loss_fucnt, device):
    running_loss = []
    running_sample_time = torch.FloatTensor().to(device)
    running_sample_event = torch.LongTensor().to(device)
    running_hazard = torch.FloatTensor().to(device)

    model.eval()
    with torch.no_grad():
        for x_modal, data_label, mask in dataloader:
            for modality in x_modal:
                x_modal[modality] = x_modal[modality].to(device)
            event = data_label[:, 0].to(device)
            time = data_label[:, 1].to(device)
            mask = mask.to(device)
            hazard, representation = model(x_modal, mask)
            loss = loss_fucnt(representation, list(representation.keys()), hazard, event, time)
            hazard = hazard['hazard']
            
            running_loss.append(loss.item())
            running_sample_time = torch.cat((running_sample_time, time.data.float()))
            running_sample_event = torch.cat((running_sample_event, event.long().data))
            running_hazard = torch.cat((running_hazard, hazard.detach()))
            
    eval_loss = torch.mean(torch.tensor(running_loss))
    eval_c_index = concordance_index(running_sample_time.cpu().numpy(), -running_hazard.cpu().numpy(), running_sample_event.cpu().numpy())
    return eval_loss, eval_c_index

In [5]:
import copy
import random
import logging 
import numpy as np
from utils.loss import Loss
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.dataset import MultiModalDataset
from utils.encoder import EncoderModel


def save_model(model, path):
    torch.save(model.state_dict(), path)

def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
        
def train_eval_model(args):
    set_seed(args.seed)
    os.makedirs(args.result_path, exist_ok=True)

    logging.basicConfig(format='%(message)s')
    for k in range(args.fold_range):
        args.fold = k
        args.log_file_name = f"result_fold{args.fold}.log"
        args.log_file_name = os.path.join(args.result_path, args.log_file_name)
        if os.path.exists(args.log_file_name):
            os.remove(args.log_file_name)
            
        handler = logging.FileHandler(args.log_file_name, mode='w')
        logger=logging.getLogger() 
        logger.setLevel(logging.DEBUG) 
        logger.addHandler(handler)
        
        train_dataset = MultiModalDataset(args, 'train', args.modalities, args.modality_data_path, remove_missing=args.remove_missing)
        train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers)
        test_dataset = MultiModalDataset(args, 'test', args.modalities, args.modality_data_path)
        test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers)

        device = torch.device(args.device)
        encoder_model = EncoderModel(args.modalities, args.modality_fv_len, args.input_modality_dim)
        model = AggModel(args.num_modalities, args.modality_fv_len, encoder_model)
        optimizer = Adam(model.parameters(), lr=args.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(  
            optimizer=optimizer, mode='max', factor=0.1,
            patience = args.scheduler_patience, verbose=True, threshold=1e-3,
            threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-8)
        model.to(device)
        
        loss_fucnt = Loss(trade_off=args.loss_trade_off, mode=args.loss_mode)

        for epoch in range(args.epochs):
            running_loss = []
            running_sample_time = torch.FloatTensor().to(device)
            running_sample_event = torch.LongTensor().to(device)
            running_hazard = torch.FloatTensor().to(device)
            
            model.train()
            for x_modal, data_label, mask in train_dataloader:
                for modality in x_modal:
                    x_modal[modality] = x_modal[modality].to(device)
                event = data_label[:, 0].to(device)
                time = data_label[:, 1].to(device)
                mask = mask.to(device)
                hazard, representation = model(x_modal, mask)
                loss = loss_fucnt(representation, list(representation.keys()), hazard, event, time)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                hazard = hazard['hazard']
                
                running_loss.append(loss.item())
                running_sample_time = torch.cat((running_sample_time, time.data.float()))
                running_sample_event = torch.cat((running_sample_event, event.long().data))
                running_hazard = torch.cat((running_hazard, hazard.detach()))
                
            train_loss = torch.mean(torch.tensor(running_loss))
            train_c_index = concordance_index(running_sample_time.cpu().numpy(), -running_hazard.cpu().numpy(), running_sample_event.cpu().numpy())
            print_log(f"Epoch {epoch}/{args.epochs}: Training Loss: {train_loss}, C-Index: {train_c_index}", logger)
            scheduler.step(train_c_index)
            if scheduler.patience == scheduler.num_bad_epochs: 
                print_log("Reducing the learning rate", logger)
            best_model = copy.deepcopy(model.state_dict())
        print_log("*" * 100, logger)
        
        model.load_state_dict(best_model)
        model_save_path = os.path.join(args.result_path, f"model_fold{args.fold}.pt")
        save_model(model, model_save_path)
        train_loss, train_c_index = evaluation(model, train_dataloader, loss_fucnt, device)
        print_log(f"Final Train Loss: {train_loss}, C-Index: {train_c_index}", logger)
        test_loss, test_c_index = evaluation(model, test_dataloader, loss_fucnt, device)
        print_log(f"Test Loss: {test_loss}, C-Index: {test_c_index}", logger)
        
        logger.removeHandler(handler)

In [6]:
import numpy as np


all_modalities = ["clinical", "miRNA", "mRNA", "WSI"]
all_modalities = np.array(all_modalities)
i = 2 ** len(all_modalities) - 1
selection = F"{i:0{len(all_modalities)}b}"
modality_select = all_modalities[np.array(list(selection)) == '1']
args = Args(modality_select)
train_eval_model(args)

Result Path: ./logs/max/results_5_24_clinical_miRNA_mRNA_WSI


Epoch 0/20: Training Loss: 4.290692329406738, C-Index: 0.6144290835841297


Epoch 0/20: Training Loss: 4.290692329406738, C-Index: 0.6144290835841297


Epoch 1/20: Training Loss: 3.859844207763672, C-Index: 0.7709260080814379


Epoch 1/20: Training Loss: 3.859844207763672, C-Index: 0.7709260080814379


...

Epoch 19/20: Training Loss: 2.5688040256500244, C-Index: 0.943210567286173
****************************************************************************************************


Final Train Loss: 2.642120599746704, C-Index: 0.9596915442249684


Final Train Loss: 2.642120599746704, C-Index: 0.9596915442249684


Test Loss: 3.7141411304473877, C-Index: 0.7670475488389237


Test Loss: 3.7141411304473877, C-Index: 0.7670475488389237
